# Activity Log

> class based approach for interacting with the activity log


In [ ]:
# | default_exp classes.DomoActivityLog

In [ ]:
# | exporti

from enum import Enum
from typing import Optional

import datetime as dt
import pandas as pd
from pprint import pprint

import aiohttp

import domolibrary.DomoAuth as dmda

import domolibrary.routes.activity_log as activity_log_routes
import domolibrary.utils.convert as convert

# from .DomoDataset import DomoDataset
# from ..utils.DictDot import DictDot

## Valid Object Types

To get an updated list of objectTypes run the `get_activity_log_object_types` function.

Note: in the `ActivityLog_ObjectType` enum, some of the entries have been renamed for clarity.


In [ ]:
# | exports


class ActivityLog_ObjectType(Enum):
    """enumerates valid object types to pass to activity log api"""

    ACCESS_TOKEN = "ACCESS_TOKEN"
    ACCOUNT = "ACCOUNT"
    ACTIVITY_LOG = "ACTIVITY_LOG"
    ALERT = "ALERT"
    APP = "APP"
    APPDB_COLLECTION = "MAGNUM_COLLECTION"
    APPDB_DATASTORE = "MAGNUM_DATASTORE"
    AUTHORITY = "AUTHORITY"
    BEAST_MODE_FORMULA = "BEAST_MODE_FORMULA"
    BUZZ_CHANNEL = "CHANNEL"
    BUZZ_GROUP_CHAT = "GROUP_CHAT"
    BUZZ_THREAD = "HUDDLE"
    CARD = "CARD"
    CHART_COLOR_PALETTE = "CHART_COLOR_PALETTE"
    COLLECTION = "COLLECTION"
    CUSTOMER = "CUSTOMER"
    CUSTOMER_STATE = "CUSTOMER_STATE"
    CUSTOMER_TIER = "CUSTOMER_TIER"
    DATA_SCIENCE_NOTEBOOK = "DATA_SCIENCE_NOTEBOOK"
    DATAFLOW = "DATAFLOW_TYPE"
    DATASET = "DATA_SOURCE"
    DATASOURCE = "DATASOURCE"
    DEPLOYMENT = "DEPLOYMENT"
    DRILL_VIEW = "DRILL_VIEW"
    EASY_INVITE_LINK = "EASY_INVITE_LINK"
    ENABLED = "ENABLED"
    FILE = "FILE"
    FILE_VERSION = "FILE_REVISION"
    GROUP = "GROUP"
    LICENSE_PAGE = "LICENSE_PAGE"
    LOGIN_SETTINGS = "LOGIN_SETTINGS"
    NAME = "NAME"
    PDP_FILTER = "ADC_FILTER"
    PDP_POLICY = "ADC_POLICY"
    PAGE = "PAGE"
    PAGE_ANALYZER = "PAGE_ANALYZER"
    PAGE_COLLECTION = "PAGE_COLLECTION"
    PROJECT = "PROJECT"
    PROJECT_LIST = "PROJECT_LIST"
    PROJECT_TASK = "PROJECT_TASK"
    PROJECT_TASK_ATTACHMENT = "PROJECT_TASK_ATTACHMENT"
    PROJECT_TASK_OWNER = "PROJECT_TASK_OWNER"
    PROXIER_EMAIL = "PROXIER_EMAIL"
    PUBLIC_EMBED_URL = "PUBLIC_URL"
    PUBLICATION = "PUBLICATION"
    REPOSITORY = "REPOSITORY"
    REPOSITORY_AUTHORIZATION = "REPOSITORY_AUTHORIZATION"
    ROLE = "ROLE"
    SEGMENT = "SEGMENT"
    SSO_PAGE = "SSO_PAGE"
    SUBSCRIBER = "PROXY_USER"
    USER = "USER"
    USER_STATE = "USER_STATE"
    VARIABLE = "VARIABLE"
    VARIABLE_CONTROL = "VARIABLE_CONTROL"
    NOTEBOOK_VIEW = "CONTAINER_VIEW"
    VIEW = "VIEW"
    VIRTUAL_USER = "VIRTUAL_USER"
    WORKBENCH_AGENT = "Workbench_AGENT"
    WORKBENCH_JOB = "Workbench_JOB"
    WORKBENCH_SCHEDULE = "Workbench_SCHEDULE"

In [ ]:
# | export
class DomoActivityLog:
    def _process_activity_log_row(row):

        if row.get('time'):
            row.update(
                {'time_dt': convert.convert_epoch_millisecond_to_datetime(row.get('time'))})

            row.update({'date': row.get("time_dt").date()})

        return row

    @classmethod
    async def get_activity_log(
        cls,
        auth: dmda.DomoAuth,
        start_time: dt.datetime,
        end_time: dt.datetime,
        object_type: Optional[ActivityLog_ObjectType] = None,
        maximum: int = 1000,
        session: Optional[aiohttp.ClientSession] = None,
        debug_api: bool = False,
        debug_loop: bool = False,
    ):
        """queries the activity log"""

        start_time_epoch = convert.convert_datetime_to_epoch_millisecond(
            start_time)
        end_time_epoch = convert.convert_datetime_to_epoch_millisecond(
            end_time)

        res_activity_log = await activity_log_routes.search_activity_log(
            auth=auth,
            start_time=start_time_epoch,
            end_time=end_time_epoch,
            maximum=maximum,
            object_type=object_type.value,
            session=session,
            debug_api=debug_api,
            debug_loop=debug_loop
        )

        if res_activity_log.is_success:
            return [cls._process_activity_log_row(row) for row in res_activity_log.response]

        return None


In [ ]:
import os
import datetime as dt

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(domo_instance='domo-dojo',
                                    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"])

al_res = await DomoActivityLog.get_activity_log(
    auth = token_auth,
    start_time = start_datetime,
    end_time = end_datetime,
    object_type = ActivityLog_ObjectType.ACTIVITY_LOG,
    maximum =10)

pd.DataFrame(al_res)

,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId,time_dt,date
0,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673052695222,Logged in,None,2023-01-07 00:51:35.222,2023-01-07
1,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673052694881,Logged in,None,2023-01-07 00:51:34.881,2023-01-07
2,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673052694218,Logged in,None,2023-01-07 00:51:34.218,2023-01-07
3,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673050902586,Logged in,None,2023-01-07 00:21:42.586,2023-01-07
4,test12@domo.com,test12@domo.com,USER,,0,FAILED_LOGIN,,,,test12@domo.com failed logging in. Logged in f...,1673050902494,Failed login,None,2023-01-07 00:21:42.494,2023-01-07
5,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673050902268,Logged in,None,2023-01-07 00:21:42.268,2023-01-07
6,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673050901659,Logged in,None,2023-01-07 00:21:41.659,2023-01-07
7,test@test.com,test@test.com,USER,,0,FAILED_LOGIN,,,,test@test.com failed logging in. Logged in fro...,1673050901573,Failed login,None,2023-01-07 00:21:41.573,2023-01-07
8,test12@domo.com,test12@domo.com,USER,,0,FAILED_LOGIN,,,,test12@domo.com failed logging in. Logged in f...,1673050553593,Failed login,None,2023-01-07 00:15:53.593,2023-01-07
9,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673050553558,Logged in,None,2023-01-07 00:15:53.558,2023-01-07


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
